<a href="https://colab.research.google.com/github/jing-mae/RoBERTa/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
!pip install torch transformers pandas scikit-learn matplotlib --quiet

import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import os
import random
import torch.nn as nn # Import nn module

# %%
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# %%
torch.manual_seed(42)
# %%
from google.colab import drive
drive.mount('/content/drive')

# 设置路径
DATA_PATH = "/content/drive/MyDrive/cleaned_final_with_tags_v5.csv"

# 读取 CSV 文件
df = pd.read_csv(DATA_PATH)
print(f"Loaded {len(df)} samples.")
print(df.head())

# %%
merge_map = {
    "neo_nazi": "white_supremacy",
    "nationalist_extremism": "white_supremacy",
    "racism": "white_supremacy",
    "xenophobia": "white_supremacy",
    "anti_semitic": "white_supremacy"
}

df['all_hate_tags'] = df['all_hate_tags'].apply(lambda tags: list(set([merge_map.get(tag, tag) for tag in eval(tags)])))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00
Using device: cuda
Mounted at /content/drive
Loaded 500 samples.
                                              lyrcis  label hate_type  \
0  Hit my snooze alarm for the twenty seventh tim...    0.0     

In [ ]:
merge_map = {
    "neo_nazi": "white_supremacy",
    "nationalist_extremism": "white_supremacy",
    "racism": "white_supremacy",
    "xenophobia": "white_supremacy",
    "anti_semitic": "white_supremacy"
}

# Assuming 'all_hate_tags' is already a list after the previous step
df['all_hate_tags'] = df['all_hate_tags'].apply(lambda tags: list(set([merge_map.get(tag, tag) for tag in tags])))

# 检查标签分布
from collections import Counter
# Corrected list comprehension
all_tags = [tag for tag_list in df['all_hate_tags'] for tag in tag_list]
tag_counts = Counter(all_tags)
print("标签分布：")
print(tag_counts)

标签分布：
Counter({'white_supremacy': 235, 'mythological_justification': 156, 'religious': 105, 'extreme_violence': 74})


In [ ]:
labels_flat = [tag for sublist in df['all_hate_tags'] for tag in sublist]
unique_labels = np.array(list(set(labels_flat)))
class_weights = compute_class_weight(class_weight='balanced', classes=unique_labels, y=labels_flat)

weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# 转换为字典
label_weights = dict(zip(unique_labels, class_weights))
print("标签权重：")
print(label_weights)


标签权重：
{np.str_('religious'): np.float64(1.3571428571428572), np.str_('mythological_justification'): np.float64(0.9134615384615384), np.str_('extreme_violence'): np.float64(1.9256756756756757), np.str_('white_supremacy'): np.float64(0.6063829787234043)}


In [ ]:
def augment_text(text):
    if isinstance(text, str):
        words = text.split()
        if len(words) > 5:
            num_to_remove = random.randint(1, 3)
            for _ in range(num_to_remove):
                words.pop(random.randint(0, len(words) - 1))
        return " ".join(words)
    return text

# 数据增强
df_augmented = df.copy()
df_augmented['lyrcis'] = df_augmented['lyrcis'].apply(augment_text)

df_combined = pd.concat([df, df_augmented], ignore_index=True)

print(f"Data after augmentation: {len(df_combined)} samples.")

Data after augmentation: 1000 samples.


In [ ]:
mlb = MultiLabelBinarizer()
multi_hot_labels = mlb.fit_transform(df_combined['all_hate_tags'])

# 重新生成 DataFrame
final_df = pd.concat([df_combined[['lyrcis']], pd.DataFrame(multi_hot_labels, columns=mlb.classes_)], axis=1)

# 划分训练集和验证集
# Filter out non-string entries from the 'lyrcis' column before splitting
valid_indices = final_df['lyrcis'].apply(lambda x: isinstance(x, str))
filtered_df = final_df[valid_indices]

train_texts, val_texts, train_labels, val_labels = train_test_split(
    filtered_df['lyrcis'].tolist(),
    filtered_df.drop(columns=['lyrcis']).values.tolist(),
    test_size=0.2,
    random_state=42
)

print(f"Train samples: {len(train_labels)}, Validation samples: {len(val_labels)}")



Train samples: 798, Validation samples: 200


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

class HateLyricsMultiLabelDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        encoding['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return encoding

train_dataset = HateLyricsMultiLabelDataset(train_texts, train_labels, tokenizer)
val_dataset = HateLyricsMultiLabelDataset(val_texts, val_labels, tokenizer)

print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Train samples: 798, Validation samples: 200


In [ ]:
from transformers import RobertaConfig, RobertaForSequenceClassification

num_labels = len(label_weights)
config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=num_labels,
    hidden_dropout_prob=0.4,
    attention_probs_dropout_prob=0.4
)

class CustomRobertaForMultiLabel(RobertaForSequenceClassification):
    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        # 只传需要的参数给父类
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        logits = outputs.logits

        # 多标签 loss 计算
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=weights_tensor)
            loss = loss_fct(logits, labels)

        return {"loss": loss, "logits": logits}

model = CustomRobertaForMultiLabel.from_pretrained("roberta-base", config=config)
model.to(device)

print(f"Model initialized with {num_labels} labels and enhanced dropout.")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of CustomRobertaForMultiLabel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model initialized with 4 labels and enhanced dropout.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=200,
    weight_decay=0.01,
    learning_rate=1e-5,
    # evaluation_strategy="epoch", # Changed to eval_strategy for compatibility with older transformers versions
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

print("Training arguments configured.")

Training arguments configured.


In [ ]:
# %%
from transformers import TrainingArguments, DataCollatorWithPadding, Trainer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score # Ensure these are imported

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = (logits > 0).astype(int)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    accuracy = accuracy_score(labels, preds)
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("Trainer initialized with custom loss function. Starting training...")

Trainer initialized with custom loss function. Starting training...


In [ ]:
trainer.train()
print("Training complete.")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mmaeve53 (mmaeve53-stockholm-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.597400,0.585380,0.395000,0.000000,0.000000,0.000000
2,0.495500,0.509039,0.500000,0.323918,0.306891,0.344041
3,0.470000,0.428723,0.510000,0.477618,0.449873,0.511754
4,0.389500,0.427148,0.520000,0.493138,0.446940,0.553846
5,0.455800,0.395876,0.560000,0.603610,0.629712,0.589103
6,0.357200,0.403740,0.565000,0.495428,0.514013,0.480390


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.597400,0.585380,0.395000,0.000000,0.000000,0.000000
2,0.495500,0.509039,0.500000,0.323918,0.306891,0.344041
3,0.470000,0.428723,0.510000,0.477618,0.449873,0.511754
4,0.389500,0.427148,0.520000,0.493138,0.446940,0.553846
5,0.455800,0.395876,0.560000,0.603610,0.629712,0.589103
6,0.357200,0.403740,0.565000,0.495428,0.514013,0.480390
7,0.400900,0.396538,0.565000,0.511367,0.572838,0.482780
8,0.396900,0.380480,0.565000,0.626322,0.618069,0.636026
9,0.342400,0.399399,0.575000,0.635926,0.617415,0.656005
10,0.367100,0.386567,0.575000,0.635910,0.626776,0.645836


Training complete.


In [ ]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.35919493436813354, 'eval_accuracy': 0.6, 'eval_f1': 0.6956272479953055, 'eval_precision': 0.675330595071128, 'eval_recall': 0.7329283216783218, 'eval_runtime': 6.0628, 'eval_samples_per_second': 32.988, 'eval_steps_per_second': 8.247, 'epoch': 20.0}


In [ ]:
# 保存模型和 tokenizer
model_path = "/content/hate_roberta_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# 打包为 zip
!zip -r hate_roberta_model.zip /content/hate_roberta_model

# 下载到本地
from google.colab import files
files.download("hate_roberta_model.zip")

print("✅ 模型已保存为本地文件 hate_roberta_model.zip")



✅ 模型保存成功


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/hate_roberta_model /content/drive/MyDrive/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

model_path = "/content/hate_roberta_model"
print("📂 文件列表：")
print(os.listdir(model_path))


📂 文件列表：
['model.safetensors', 'vocab.json', 'tokenizer_config.json', 'config.json', 'merges.txt', 'special_tokens_map.json']


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving eva_final.csv to eva_final (1).csv


In [ ]:
df_eval = pd.read_csv("eva_final.csv")
df_eval = df_eval.dropna(subset=['lyrics', 'label'])
print(f"Loaded {len(df_eval)} evaluation samples.")


Loaded 150 evaluation samples.


In [ ]:
import pandas as pd

# ✅ 加载测试数据
df_eval = pd.read_csv("eva_final.csv")

# ✅ 合并标签列为列表
def collect_tags(row):
    return list(filter(pd.notna, [row['hate_label'], row['hate_label2'], row['hate_label3']]))

df_eval['all_tags'] = df_eval.apply(collect_tags, axis=1)


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# 所有可能的标签（必须和训练时一致）
label_classes = ['white_supremacy', 'religious', 'extreme_violence', 'mythological_justification']

mlb = MultiLabelBinarizer(classes=label_classes)
true_binary = mlb.fit_transform(df_eval["all_tags"])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
model_path = "/content/drive/MyDrive/hate_roberta_model"

from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained(model_path, local_files_only=True)
model = RobertaForSequenceClassification.from_pretrained(model_path, local_files_only=True)
model.to("cuda" if torch.cuda.is_available() else "cpu")


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.4, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.4, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
# 预处理歌词
lyrics = df_eval["lyrics"].tolist()

encodings = tokenizer(
    lyrics,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

encodings = {k: v.to(model.device) for k, v in encodings.items()}

# 预测
with torch.no_grad():
    outputs = model(**encodings)
    logits = outputs.logits
    probs = torch.sigmoid(logits).cpu().numpy()

# 将概率转换为 0/1 标签
threshold = 0.5
predictions = (probs >= threshold).astype(int)


In [ ]:
from sklearn.metrics import classification_report

print("📊 Evaluation Report:")
print(classification_report(true_binary, predictions, target_names=mlb.classes_))


📊 Evaluation Report:
                            precision    recall  f1-score   support

           white_supremacy       0.94      0.61      0.74        56
                 religious       0.13      0.80      0.23         5
          extreme_violence       0.33      0.38      0.36        21
mythological_justification       0.13      1.00      0.23         8

                 micro avg       0.36      0.60      0.45        90
                 macro avg       0.39      0.70      0.39        90
              weighted avg       0.68      0.60      0.58        90
               samples avg       0.13      0.21      0.16        90



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
